In [ ]:
import pandas as pd
import numpy as np
import glob
import os

# === Load and Combine Tick Data ===
folder = "18-07-2025_0_extracted"
files = glob.glob(os.path.join(folder, "*.txt"))

columns = [
    "symbol", "open", "last", "high", "low", "change", "bPrice", "bQty", "sPrice", "sQty",
    "ltq", "avgPrice", "quotes", "ttq", "totalBuyQt", "totalSellQ", "ttv", "trend",
    "lowerCktLm", "upperCktLm", "ltt", "close", "exchange", "stock_name", "VolumeC", "OI",
    "CHNGOI", "product_type", "expiry_date", "strike_price"
]

df_list = []
for file in files:
    df_temp = pd.read_csv(file, header=None, names=columns)
    df_list.append(df_temp)

df = pd.concat(df_list, ignore_index=True).drop_duplicates()
numeric_cols = ['open', 'last', 'high', 'low', 'bPrice', 'sPrice', 'bQty', 'sQty', 'ltq', 'VolumeC']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# === Tick-based Buy/Sell Pressure ===
df['delta_price'] = df['last'].diff()
df['buy_volume'] = np.where(df['delta_price'] > 0, df['ltq'], 0)
df['sell_volume'] = np.where(df['delta_price'] < 0, df['ltq'], 0)

total_buy = df['buy_volume'].sum()
total_sell = df['sell_volume'].sum()

# === Order Imbalance ===
df['order_imbalance'] = (df['bQty'] - df['sQty']) / (df['bQty'] + df['sQty'])
net_order_imbalance = df['order_imbalance'].mean()

# === Money Flow Index Proxy ===
df['typical_price'] = (df['high'] + df['low'] + df['last']) / 3
df['money_flow'] = df['typical_price'] * df['VolumeC']
df['raw_flow'] = np.where(df['typical_price'] > df['typical_price'].shift(), df['money_flow'], -df['money_flow'])
mfi_score = df['raw_flow'].rolling(window=14).sum().iloc[-1]

# === Accumulation/Distribution ===
df['clv'] = ((df['last'] - df['low']) - (df['high'] - df['last'])) / (df['high'] - df['low'])
df['clv'] = df['clv'].fillna(0)
df['ad_volume'] = df['clv'] * df['VolumeC']
ad_trend = df['ad_volume'].cumsum().iloc[-1]

# === Sentiment Decision ===
print("\n💰 MONEY FLOW SENTIMENT ANALYSIS:")
print(f"Total Buy Volume: {total_buy}")
print(f"Total Sell Volume: {total_sell}")
print(f"Order Imbalance: {net_order_imbalance:.2f}")
print(f"MFI Score (14-tick): {mfi_score:.2f}")
print(f"Accumulation Trend: {ad_trend:.2f}")

if total_buy > total_sell and net_order_imbalance > 0.3 and mfi_score > 0:
    print("📈 Money Flow Sentiment: STRONG BUYING")
elif total_sell > total_buy and net_order_imbalance < -0.3 and mfi_score < 0:
    print("📉 Money Flow Sentiment: STRONG SELLING")
else:
    print("⚖️ Money Flow Sentiment: NEUTRAL / MIXED")
